In [1]:
import sys
sys.path.append('/content')
from src.fluvius import USGS_Water_DB

%load_ext autoreload
%autoreload 2

db = USGS_Water_DB()
db.get_station_df()
db.station_df



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
There is no [linux64] chromedriver for browser 90.0.4430 in cache
Get LATEST driver version for 90.0.4430
Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_linux64.zip
Driver has been saved in cache [/root/.wdm/drivers/chromedriver/linux64/90.0.4430.24]


,site_no,site_name,Latitude,Longitude,geometry
0,01472157,"French Creek near Phoenixville, PA",40.1515,-75.6013,POINT (-75.60130 40.15150)
1,01478245,"White Clay Creek near Strickersville, PA",39.7475,-75.7708,POINT (-75.77080 39.74750)
2,01480617,"West Branch Brandywine Creek at Modena, PA",39.9618,-75.8013,POINT (-75.80130 39.96180)
3,01480870,East Branch Brandywine Creek below Downingtown...,39.9687,-75.6733,POINT (-75.67330 39.96870)
4,01481000,"Brandywine Creek at Chadds Ford, PA",39.8698,-75.5933,POINT (-75.59330 39.86980)
...,...,...,...,...,...
160,393806095273700,"Atchison County Lake near Horton, KS",39.6350,-95.4603,POINT (-95.46030 39.63500)
161,393817095260100,"Clear Creek at Decator Road near Horton, KS",39.6381,-95.4336,POINT (-95.43360 39.63810)
162,394126096073500,Black Vermillion River Tributary above Central...,39.6906,-96.1264,POINT (-96.12640 39.69060)
163,394146096085500,"Centralia Lake near Centralia, KS",39.6961,-96.1486,POINT (-96.14860 39.69610)


In [1]:
### Environment setup
from pystac_client import Client
import planetary_computer as pc
import os

# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# pc.settings.set_subscription_key(<YOUR API Key>)
env_vars = !cat /content/.env

for var in env_vars:
    key, value = var.split(' = ')
    os.environ[key] = value

In [16]:
import pandas as pd
import fsspec
import numpy as np
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt

container = 'itv-data'

storage_options={'account_name':os.environ['ACCOUNT_NAME'],\
                 'account_key':os.environ['BLOB_KEY']}
fs = fsspec.filesystem('az', account_name=storage_options['account_name'], account_key=storage_options['account_key'])
#station_url = f'az://{container}/station_metadata.csv'
#station_df = pd.read_csv(station_url, storage_options=storage_options) 

#img_station_url = f'az://{container}/station_img_metadata.csv'
#img_df = pd.read_csv(img_station_url, storage_options=storage_options)

In [18]:
datalist = fs.ls(f'{container}')
ds = [d for d in datalist if '.csv' in d]

In [108]:
import bokeh
from bokeh import palettes
import math
n = 7
#vmax = 100 #canopy
#vmax = 712 #basal
vmax = 685 #biomass
significant_digits = 1
r_vmax =  round(vmax, significant_digits - int(math.floor(math.log10(abs(vmax)))) - 1)
_div = r_vmax/100*np.array([  0,   1,   5,  10,  20,  50, 100])[::-1]
p = list(palettes.viridis(n))
p.reverse()
#_div = np.linspace(r_vmax,0,n) 
cbar = {c:int(v) for c,v in zip(p,_div)}


In [115]:
np.linspace(0,1,8).round(2)*100

array([  0.,  14.,  29.,  43.,  57.,  71.,  86., 100.])

{'#440154': 0,
 '#404387': 20,
 '#29788E': 40,
 '#22A784': 60,
 '#79D151': 80,
 '#FDE724': 100}

palettes.viridis(6)

## 500 mb per 3 channel image on sentinel-2 including scl downsampled

In [12]:
total_tb = np.sum(img_df.cloudless_img_count)*500/1000000

In [17]:
print(f'USGS total memory needed for {np.sum(img_df.cloudless_img_count)} images: {total_tb} TB')

USGS total memory needed for 3564 images: 1.782 TB


In [18]:
#this does not include prediction

In [21]:
ana = 29*500/1000000

In [23]:
total_tb + ana

1.7965